In [1]:
# Imports
import os
import json
import numpy as np
import pandas as pd

# Nuestro path de inicio siempre debe ser el home dir
os.chdir(os.path.abspath(".."))

In [8]:
# Archivos importantes y paths
df_annotations = pd.read_excel('data/annotations.xlsx')
dir_trans = 'data/TRANSCRIPCIONES/WHISPER'

In [26]:
# Obtengo todos los audios diferents y defino un diccionario vacío
audios_name = df_annotations['Audio_Name'].unique()
trans_dict = {}

# Loop para optener todas las tuplas de inicio fin de todas las transcripciones y guardarlas en el diccionario trans_dict
for audio_name in audios_name:
    audio_name_json = audio_name[:-4] + '.json'
    
    with open(f'{dir_trans}/{audio_name_json}') as f:
        audio_data = json.load(f)
    
    trans_dict[audio_name] = {}
    x = []
    y = []
    for segment in audio_data['segments']:
        x.append((segment['start'], segment['end']))
        y.append(segment['text'])
    
    trans_dict[audio_name]['rangos'] = x
    trans_dict[audio_name]['texto'] = y

In [27]:
# Verificado estan los 244 audios
print(len(trans_dict.keys()))
print(trans_dict.keys())

244
dict_keys(['MSP-Conversation_0021.wav', 'MSP-Conversation_0023.wav', 'MSP-Conversation_0035.wav', 'MSP-Conversation_0047.wav', 'MSP-Conversation_0061.wav', 'MSP-Conversation_0079.wav', 'MSP-Conversation_0083.wav', 'MSP-Conversation_0087.wav', 'MSP-Conversation_0088.wav', 'MSP-Conversation_0103.wav', 'MSP-Conversation_0114.wav', 'MSP-Conversation_0125.wav', 'MSP-Conversation_0135.wav', 'MSP-Conversation_0147.wav', 'MSP-Conversation_0153.wav', 'MSP-Conversation_0160.wav', 'MSP-Conversation_0166.wav', 'MSP-Conversation_0167.wav', 'MSP-Conversation_0172.wav', 'MSP-Conversation_0180.wav', 'MSP-Conversation_0190.wav', 'MSP-Conversation_0201.wav', 'MSP-Conversation_0202.wav', 'MSP-Conversation_0218.wav', 'MSP-Conversation_0227.wav', 'MSP-Conversation_0231.wav', 'MSP-Conversation_0257.wav', 'MSP-Conversation_0260.wav', 'MSP-Conversation_0261.wav', 'MSP-Conversation_0269.wav', 'MSP-Conversation_0281.wav', 'MSP-Conversation_0295.wav', 'MSP-Conversation_0300.wav', 'MSP-Conversation_0361.wav',

In [28]:
# Veo cantidad de segmentos por audio
for i in trans_dict.keys():
    print(i, len(trans_dict[i]['rangos']))

MSP-Conversation_0021.wav 1364
MSP-Conversation_0023.wav 148
MSP-Conversation_0035.wav 201
MSP-Conversation_0047.wav 207
MSP-Conversation_0061.wav 152
MSP-Conversation_0079.wav 278
MSP-Conversation_0083.wav 170
MSP-Conversation_0087.wav 357
MSP-Conversation_0088.wav 248
MSP-Conversation_0103.wav 914
MSP-Conversation_0114.wav 340
MSP-Conversation_0125.wav 392
MSP-Conversation_0135.wav 296
MSP-Conversation_0147.wav 141
MSP-Conversation_0153.wav 544
MSP-Conversation_0160.wav 384
MSP-Conversation_0166.wav 262
MSP-Conversation_0167.wav 319
MSP-Conversation_0172.wav 402
MSP-Conversation_0180.wav 226
MSP-Conversation_0190.wav 283
MSP-Conversation_0201.wav 259
MSP-Conversation_0202.wav 121
MSP-Conversation_0218.wav 252
MSP-Conversation_0227.wav 270
MSP-Conversation_0231.wav 315
MSP-Conversation_0257.wav 227
MSP-Conversation_0260.wav 388
MSP-Conversation_0261.wav 185
MSP-Conversation_0269.wav 492
MSP-Conversation_0281.wav 385
MSP-Conversation_0295.wav 211
MSP-Conversation_0300.wav 321
MSP-Conve

In [29]:
# único audio que llama la atención, pero esta en development
df_annotations[df_annotations['Audio_Name'] == 'MSP-Conversation_0653.wav'].head()

Audio_Name        Conversation_Part  \
1977  MSP-Conversation_0653.wav  MSP-Conversation_0653_1   
1978  MSP-Conversation_0653.wav  MSP-Conversation_0653_1   
1979  MSP-Conversation_0653.wav  MSP-Conversation_0653_1   
1980  MSP-Conversation_0653.wav  MSP-Conversation_0653_1   
1981  MSP-Conversation_0653.wav  MSP-Conversation_0653_1   

                      Annotation_File  Emotion  Annotator  PC_Num  Part_Num  \
1977  MSP-Conversation_0653_1_002.csv  Arousal          2     653         1   
1978  MSP-Conversation_0653_1_005.csv  Arousal          5     653         1   
1979  MSP-Conversation_0653_1_006.csv  Arousal          6     653         1   
1980  MSP-Conversation_0653_1_007.csv  Arousal          7     653         1   
1981  MSP-Conversation_0653_1_008.csv  Arousal          8     653         1   

             Type  start_time  end_time  
1977  Development         0.0   322.775  
1978  Development         0.0   322.775  
1979  Development         0.0   322.775  
1980  Development         0.0   322.775  
1981  Development         0.0   322.775

In [30]:
# Guardo el diccionario en archivo json
with open('data/intervalos_transcripciones_whisper.json', 'w') as f:
    json.dump(trans_dict, f)

Estadística de duración promedio de segmentos

In [31]:
x = []
segmentos = []

for key in trans_dict:
    
    segmentos.append(len(trans_dict[key]['rangos']))
    
    for segmento in trans_dict[key]['rangos']:
        fin = segmento[1]
        inicio = segmento[0]
        
        x.append(fin - inicio)
        
print('Segmentos por audio:', np.mean(segmentos))
print('Duración promedio de segmento:', np.mean(x))

Segmentos por audio: 335.79918032786884
Duración promedio de segmento: 2.3928943674864223


Verficación manual de segmentos

In [63]:
from pydub import AudioSegment
from pydub.playback import play


rango = trans_dict['MSP-Conversation_1568.wav']['rangos'][35]
texto = trans_dict['MSP-Conversation_1568.wav']['texto'][35]
audio = AudioSegment.from_file("data/MSPCORPUS/Audio/MSP-Conversation_1568.wav")

start_time = (rango[0])* 1000 # 10 seconds
end_time = (rango[1] + 0.1) * 1000 # 3 seconds after the start

segment = audio[start_time:end_time]

print(rango)
print(texto)
play(segment)

(89.8, 93.06)
 You know, they say the smartest things and you want to call everyone.
